In [ ]:
# Cell 1: Setup and Imports
from langchain.chains import LLMChain
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
import logging
import json
from datetime import datetime
from pathlib import Path
import io

# Set up logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler('story_generation.log'),
        logging.StreamHandler()
    ]
)

# Create timestamp and directory for this session
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
save_dir = Path(f"story_generation_{timestamp}")
save_dir.mkdir(exist_ok=True)

In [ ]:
# Cell 2: Initialize Models
def create_llms():
    return {
        'experience': ChatOpenAI(temperature=0.7),
        'story': ChatOpenAI(temperature=0.9),
        'reflection': ChatOpenAI(temperature=0.4),
        'refinement': ChatOpenAI(temperature=0.7),
        'latex': ChatOpenAI(temperature=0.3)
    }

def create_prompts():
    return {
        'experience': PromptTemplate(
            input_variables=["context"],
            template="""Given these story elements and memories:
            
            {context}
            
            Suggest deeply human experiences, emotions, and universal themes that relate to these elements. 
            Consider fundamental human experiences like love, loss, growth, fear, triumph, etc.
            Be specific and draw meaningful connections."""
        ),
        'story': PromptTemplate(
            input_variables=["experiences", "related_memories"],
            template="""Using these human experiences and thematic elements:
            
            {experiences}
            
            And drawing from these related memories and story elements:
            
            {related_memories}
            
            Craft a rich and engaging story segment that weaves these elements together.
            Focus on vivid imagery, emotional resonance, and narrative flow."""
        ),
        'reflection': PromptTemplate(
            input_variables=["story", "experiences"],
            template="""Reflect on this story segment and its themes:
            
            {story}
            
            Consider these underlying experiences and emotions:
            {experiences}
            
            Analyze the narrative for:
            1. Thematic consistency
            2. Character development
            3. Emotional resonance
            4. Plot coherence
            5. Symbolic depth
            
            Provide specific suggestions for deepening and enriching the narrative."""
        ),
        'refinement': PromptTemplate(
            input_variables=["story", "reflection", "experiences"],
            template="""Enhance this story segment:
            
            {story}
            
            Based on this reflection:
            {reflection}
            
            And maintaining these core experiences:
            {experiences}
            
            Rewrite the segment with deeper emotional resonance, richer symbolism, and stronger narrative cohesion."""
        ),
        'latex': PromptTemplate(
            input_variables=["story"],
            template="""Convert this story segment into properly formatted LaTeX:
            
            {story}
            
            Use appropriate LaTeX formatting for literary text, including proper paragraph breaks,
            quotation marks, and any needed structural elements."""
        )
    }

In [ ]:
# Cell 3: Story Processing Functions
class StoryProcessor:
    def __init__(self, save_dir):
        self.llms = create_llms()
        self.prompts = create_prompts()
        self.chains = self._create_chains()
        self.save_dir = Path(save_dir)
    
    def _create_chains(self):
        return {
            name: LLMChain(llm=llm, prompt=self.prompts[name])
            for name, llm in self.llms.items()
        }
    
    def generate_experiences(self, context, chunk_id=None):
        logging.info("Generating experiences")
        experiences = self.chains['experience'].run(context=context)
        
        if chunk_id is not None:
            self._save_step(chunk_id, 'experiences', {'experiences': experiences})
        return experiences
    
    def generate_initial_story(self, experiences, related_memories, chunk_id=None):
        logging.info("Generating initial story")
        story = self.chains['story'].run(
            experiences=experiences,
            related_memories=related_memories
        )
        
        if chunk_id is not None:
            self._save_step(chunk_id, 'initial_story', {
                'experiences': experiences,
                'story': story
            })
        return story
    
    def generate_reflection(self, story, experiences, chunk_id=None):
        logging.info("Generating reflection")
        reflection = self.chains['reflection'].run(
            story=story,
            experiences=experiences
        )
        
        if chunk_id is not None:
            self._save_step(chunk_id, 'reflection', {'reflection': reflection})
        return reflection
    
    def refine_story(self, story, reflection, experiences, chunk_id=None):
        logging.info("Refining story")
        refined = self.chains['refinement'].run(
            story=story,
            reflection=reflection,
            experiences=experiences
        )
        
        if chunk_id is not None:
            self._save_step(chunk_id, 'refined_story', {
                'original': story,
                'reflection': reflection,
                'refined': refined
            })
        return refined
    
    def convert_to_latex(self, story, chunk_id=None):
        logging.info("Converting to LaTeX")
        latex = self.chains['latex'].run(story=story)
        
        if chunk_id is not None:
            self._save_step(chunk_id, 'latex', {'latex': latex})
        return latex
    
    def _save_step(self, chunk_id, step_name, data):
        filepath = self.save_dir / f"chunk_{chunk_id}_{step_name}.json"
        with open(filepath, 'w') as f:
            json.dump(data, f, indent=2)

In [ ]:
# Cell 4: Interactive Story Development
# Initialize processor
processor = StoryProcessor(save_dir)

# Example usage for a single chunk
chunk_id = 0  # You can change this for different chunks

# You can now use these functions interactively
context = """Your story context here"""
related_memories = """Your related memories here"""

# Generate experiences
experiences = processor.generate_experiences(context, chunk_id)
print("Generated experiences:", experiences)

# Generate initial story
initial_story = processor.generate_initial_story(experiences, related_memories, chunk_id)
print("\nInitial story:", initial_story)

# Generate reflection
reflection = processor.generate_reflection(initial_story, experiences, chunk_id)
print("\nReflection:", reflection)

# Refine story
refined_story = processor.refine_story(initial_story, reflection, experiences, chunk_id)
print("\nRefined story:", refined_story)

# Convert to LaTeX
latex_version = processor.convert_to_latex(refined_story, chunk_id)
print("\nLaTeX version:", latex_version)

In [ ]:
# Cell 5: Batch Processing (Optional)
def process_all_documents(processor, all_docs):
    output_file = io.StringIO()
    
    for current_idx, current_doc in enumerate(all_docs):
        logging.info(f"Processing document chunk {current_idx + 1}/{len(all_docs)}")
        
        current_content = current_doc.page_content
        related_docs = [doc for doc in all_docs[:current_idx]]
        related_memories = "\n\n".join([doc.page_content for doc in related_docs])
        
        # Full generation pipeline
        experiences = processor.generate_experiences(current_content, current_idx)
        initial_story = processor.generate_initial_story(experiences, related_memories, current_idx)
        reflection = processor.generate_reflection(initial_story, experiences, current_idx)
        refined_story = processor.refine_story(initial_story, reflection, experiences, current_idx)
        latex_text = processor.convert_to_latex(refined_story, current_idx)
        
        output_file.write(latex_text + "\n\n")
    
    final_latex = output_file.getvalue()
    with open(processor.save_dir / "complete_story.tex", 'w') as f:
        f.write(final_latex)
    
    return final_latex

# Example usage:
# final_story = process_all_documents(processor, all_docs)